# Multiscale particle-particle interaction

The simple particle-particle interaction example had no support for multiscale particles where particles reside on different levels. Consequently, it also did not really support adaptive meshes. This might not have been obvious, as we started with a regular particle configuration and invariant cut-off radii, but it became a problem after a while when the particles start to cluster.

In [ ]:

import os

import peano4
import peano4.dastgen2
import peano4.toolbox
import peano4.toolbox.particles
import dastgen2


In [ ]:
project = peano4.Project( ["examples", "particles"], "multiscale-particle-interaction", "." )

In [ ]:
#build_mode = peano4.output.CompileMode.Asserts
build_mode = peano4.output.CompileMode.Release
project.output.makefile.parse_configure_script_outcome( "../../.." )
project.output.makefile.set_mode( build_mode )

## Model the particle

No changes anywhere here.

In [ ]:
particle  = peano4.toolbox.particles.Particle( "Particle" )
particle.data.add_attribute( peano4.dastgen2.Peano4DoubleArray("v","Dimensions") )
particles = peano4.toolbox.particles.ParticleSet( particle )

In [ ]:
project.datamodel.add_global_object(particle)
project.datamodel.add_vertex(particles)

## Model the setup phases

The setup remains almost unchanged. I give all particles a biased random initial velocity. This will introduce an adaptive mesh, as particles will cluster in the bottom left part of the 2d domain.

In [ ]:
mesh_size = 0.2

particle_tree_analysis = peano4.toolbox.particles.ParticleTreeAnalysis(particles)
project.datamodel.add_cell(particle_tree_analysis.cell_marker)   # read docu of ParticleTreeAnalysis

create_grid = peano4.solversteps.Step( name="CreateGrid", add_user_defined_actions=False )
create_grid.add_action_set( peano4.toolbox.CreateRegularGrid(mesh_size) )
create_grid.use_vertex(particles)
create_grid.use_cell(particle_tree_analysis.cell_marker)
project.solversteps.add_step(create_grid)

In [ ]:
initialisation_snippet = """
 // we could do something intelligent here, but we actually
 // just set the cut-off radius
 particle->setCutOffRadius(0.001);
 particle->setV(0,-((double) rand() / (RAND_MAX)));
 particle->setV(1,-((double) rand() / (RAND_MAX)));
"""

init_setup = peano4.solversteps.Step( name="Init", add_user_defined_actions=False )
init_setup.use_vertex(particles)
init_setup.use_cell(particle_tree_analysis.cell_marker)
init_setup.add_action_set( peano4.toolbox.particles.UpdateParticleGridAssociation(particles) )
init_setup.add_action_set( peano4.toolbox.particles.InsertRandomParticlesIntoUnrefinedCells(
  particle_set=particles,
  average_distance_between_particles=mesh_size/10,
  initialisation_call=initialisation_snippet,
  noise=True ))
init_setup.add_action_set( particle_tree_analysis )
project.solversteps.add_step(init_setup)

In [ ]:
print_solution = peano4.solversteps.Step( "Plot", add_user_defined_actions=False )
print_solution.use_vertex(particles)
print_solution.use_cell(particle_tree_analysis.cell_marker)
print_solution.add_action_set( particle_tree_analysis )
print_solution.add_action_set( peano4.toolbox.PlotGridInPeanoBlockFormat( filename="grid", time_stamp_evaluation=peano4.toolbox.PlotGridInPeanoBlockFormat.CountTimeSteps ) )
print_solution.add_action_set( peano4.toolbox.particles.PlotParticlesInVTKFormat( "particles", particles, time_stamp_evaluation=peano4.toolbox.particles.PlotParticlesInVTKFormat.CountTimeSteps ) )
project.solversteps.add_step(print_solution)


## Particle behaviour

We stick to exactly the same particle behaviour from a code point of view. However, we use a pre-manufactured toolbox  into which we inject the actual code. The important fact here is that we now do not simply rely on the particles of the adjacent vertices of a cell. Instead, we work with active and local particle sets.

In [ ]:
from IPython.display import Image
!ls ../../python/peano4/toolbox/particles
Image("../../../python/peano4/toolbox/particles/dependency_sets.png")

To understand what the individual particles in the sketch above are supposed to mean and how they are maintained and exposed to the user, please consult the documentation of the ParticleParticleInteraction class in the Python toolbox directory. It contains a lot explanations.

In [ ]:
move_particles = peano4.solversteps.Step( "Move", add_user_defined_actions=False )
move_particles.use_vertex(particles)
move_particles.use_cell(particle_tree_analysis.cell_marker)
move_particles.add_action_set( peano4.toolbox.particles.UpdateParticleGridAssociation(particles) )
move_particles.add_action_set( peano4.toolbox.particles.ParticleParticleInteraction(
  particle_set=particles,
  cell_compute_kernel="""
if ( marker.isContained( localParticle->getX() ) ) {
  tarch::la::Vector<Dimensions,double> dist = activeParticle->getX() - localParticle->getX();
  if ( tarch::la::greater( tarch::la::norm2(dist),0.0 ) ) {
    const double mass1 = localParticle->getCutOffRadius();   // made-up hack, no real physics. But illustrates principle
    const double mass2 = activeParticle->getCutOffRadius();
    double forceQuantity = mass1 * mass2 / tarch::la::norm2(dist) / tarch::la::norm2(dist);
    localParticle->setV(
      localParticle->getV() + 0.001 * forceQuantity / tarch::la::norm2(dist) * dist
    );
  }
}
""",
  touch_vertex_first_time_compute_kernel="localParticle->setMoveState( globaldata::Particle::MoveState::NotMovedYet );",
  touch_vertex_last_time_compute_kernel="""

  const double timeStepSize = 0.0001;
  if (
    localParticle->getMoveState()!=globaldata::Particle::MoveState::Moved
    and
    localParticle->getParallelState()==globaldata::Particle::ParallelState::Local
  ) {
    localParticle->setX( localParticle->getX() + timeStepSize * localParticle->getV() );
    for (int d=0; d<Dimensions; d++) {
      if ( localParticle->getX()(d)<0.0 ) {
        localParticle->setV(d, std::abs(localParticle->getV()(d)));
      }
      if ( localParticle->getX()(d)>1.0 ) {
        localParticle->setV(d, -std::abs(localParticle->getV()(d)));
      }
    }
    localParticle->setMoveState( globaldata::Particle::MoveState::Moved );
  }
"""
  ))
move_particles.add_action_set( peano4.toolbox.particles.ParticleAMR(particles,particle_tree_analysis,min_particles_per_cell=10) )
move_particles.add_action_set( particle_tree_analysis )
project.solversteps.add_step(move_particles)

## Generate the actual C++ code

Next, we generate the actual Peano 4 C++ code:

In [ ]:
project.generate()

## Write the main

This time, the main is slightly more sophisticated. We continue to set up the grid as before, but then we always run 10 iterations of the Move, i.e. we do 10 time steps, and then we plot. The whole sequence is repeated a couple of times. Have a look into the main file in the repo to see the whole snippet:

    ...
    for (int i=0; i<100; i++) {
      for (int j=0; j<10; j++) {
        examples::particles::observers::Move moveObserver;
        peano4::parallel::SpacetreeSet::getInstance().traverse(moveObserver);
      }
      logInfo( "main()", "plot" )
      examples::particles::observers::Plot plotTimeStepObserver;
      peano4::parallel::SpacetreeSet::getInstance().traverse(plotTimeStepObserver);
    }


## Write the actual particle movement

We will implement the actual particle movement ourselves. For this, we have asked Move to be an action set with manual user code, and indeed we get a Move class in the actions subdirectory once we have triggered the build process. This is, same as main, an empty blueprint. We can alter it, and Peano won't overwrite it anymore after that. If we delete it, the API will regenerate it however. The current example has already a pre-prepared Move, so it won't be empty, but you might want to delete it and regenerate it to see what you get in the first place.

Here's what I changed in the Move class: Move is a standard class which defines an action set: There's a fixed number of callbacks that are invoked by Peano, and we can use these to inject behaviour. I only alter (or plug into) three actions: 

### First access to vertex

Particles are associated to vertices. So when I hit a vertex the first time, I also hit the particles associated with it for the first time. I run over them, and set their status to NotMovedYet:

     for (auto& p: fineGridVertexParticleSet) {
       p->setMoveState( globaldata::Particle::MoveState::NotMovedYet );
     }

### Hit cell

When I hit a cell, I actually evaluate the "forces" and alter the particles' velocities. I do this only for the particles that reside within the cell, as I know that a neighbouring cell will either be hit after or before the current one. If I modified all particles associated with the 2^d vertices of a cell, I'd modify particles multiple times. 

Please note that I only change the particles' attributes. I do not change their position. That means, the logical topology of all data, i.e. how they are assigned to vertices remains valid and intact. This means that we don't care how Peano runs through the cells. All data remains valid.

Please see touchCellFirstTime() for details.


### Last access to vertex

When I hit a vertex for the last time, I know that all 2^d adjacent cells have been "visited". Therefore, I can now finally update the vertex position. The vertex won't be used by neighbouring cells anymore and we are fine. Again, I do this only for particles that haven't moved yet. Otherwise we might move a particle, it ends up around another vertex that we haven't processed yet and then it is moved again.

    const double timeStepSize = 0.0001;
    for (auto& p: fineGridVertexParticleSet) {
      if (
        p->getMoveState()!=globaldata::Particle::MoveState::Moved
        and
        p->getParallelState()==globaldata::Particle::ParallelState::Local
      ) {
        p->setX( p->getX() + timeStepSize * p->getV() );
        for (int d=0; d<Dimensions; d++) {
          if ( p->getX()(d)<0.0 ) {
            p->setV(d, std::abs(p->getV()(d)));
          }
          if ( p->getX()(d)>1.0 ) {
            p->setV(d, -std::abs(p->getV()(d)));
          }
        }
      }
      p->setMoveState( globaldata::Particle::MoveState::Moved );
    }

You see, we also add reflecting boundary conditions.


## Build the code

In [ ]:
project.build()

## Run code

In [ ]:
output_files = [ f for f in os.listdir(".") if f.endswith(".peano-patch-file") or f.endswith(".vtu") or f.endswith(".pvd") ]
for f in output_files:
  os.remove(f)


In [ ]:
# success = project.run( args=["--threads", "1"], prefix=["mpirun", "-n", "1"] )
!./peano4

# Wrap-up

Our introductions here follow a standard Peano development pattern. Very often, we write codes via the right action sets. Once these have converged and are mature, we refactor them out into the Python API as toolset. That's how the whole particle toolset emerged. This way, they can be used easily by other users. Consequently, our next example on multiscale particles won't use a hand-written action set anymore. It will be use a pre-manufactured action set from the toolset and insert code snippets into this one.

If you study the code en detail (and let it run for a while), you'll recognise that something is wrong. This is where we need multiscale data ...